In [43]:
import pandas as pd
import glob, os
pd.set_option('float_format', '{:f}'.format)

dateparse = lambda x: pd.datetime.strptime(x, '%Y%m%d')

pd.options.display.max_columns = 999
pd.options.display.max_rows = 100

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

In [2]:
def plot_signal(df, signal_name, stop_legend = False, opacity_cndl = 0.8):
    # рисуем свечной график с объемами
    # signal_name - колонка с сигналом
    # ниже кода для трех состояний сигнала: +1, -1, 0
    # +1 синий, -1 розовый, 0 цвет свечек не меняется

    df['Date_2'] = df['Date'].dt.strftime('%d-%m-%Y')

    color_signal_long = '#0028FF' # цвет сигнала для лонга - синий
    color_signal_short = '#ff00f1' # цвет сигнала для шорта - розовый
    
    mask_long = df[signal_name]==True
#    mask_short = df[signal_name]<0
        
    name_long = 'Long_' + str(round(df[mask_long].shape[0]*100/df.shape[0],1)) + '%'
#    name_short = 'Short_' + str(round(df[mask_short].shape[0]*100/df.shape[0],1)) + '%'
        
    ticker_name = df.Ticker[0]
    data = [ dict(
        name = ticker_name + '_' + signal_name,
        type = 'candlestick',
        x = df['Date_2'],
        yaxis = 'y2',
        open = df['Open'],
        high = df['High'],
        low = df['Low'],
        close = df['Close'],
        opacity = opacity_cndl
    ),
        dict(
        name = name_long,
        type = 'candlestick',
        x = df['Date_2'][mask_long],
        yaxis = 'y2',
        open = df['Open'][mask_long],
        high = df['High'][mask_long],
        low = df['Low'][mask_long],
        close = df['Close'][mask_long],
        hoverinfo='none',
        increasing = dict(line = dict(color = '#0028FF')),
        decreasing = dict(line = dict(color = '#0028FF'))
    )#,
#        dict(
#        name = name_short,
#        type = 'candlestick',
#        x = df['Date_2'][mask_short],
 #       yaxis = 'y2',
 #       open = df['Open'][mask_short],
 #       high = df['High'][mask_short],
  #      low = df['Low'][mask_short],
  #      close = df['Close'][mask_short],
  #      hoverinfo='none',
  #      increasing = dict(line = dict(color = '#ff00f1')),
  #      decreasing = dict(line = dict(color = '#ff00f1'))
#    )
           ]

    layout=dict()

    fig = dict(data=data, layout=layout)

    fig['layout'] = dict()
    #fig['layout']['plot_bgcolor'] = 'rgb(250, 250, 250)'
    #categoryorder = 'category ascending', 
    fig['layout']['xaxis'] = dict(type ='category', 
                                  tickfont=dict(size=10),
                                  rangeselector = dict(visible = True), 
                                  rangeslider = dict(visible = False))
    fig['layout']['yaxis'] = dict(domain = [0, 0.2], 
                                  showticklabels = False, 
                                  autorange = True, 
                                  type = 'log',
#                                  title = 'Volume'
                                 )
    fig['layout']['yaxis2'] = dict(domain = [0.1, 0.95], 
                                   autorange = True, 
                                   type = 'log',
                                   title = ticker_name + ' prices')
    fig['layout']['legend'] = dict(orientation = 'h', 
                                   y = 0.95, x = 0.1, 
                                   yanchor = 'bottom')
    fig['layout']['margin'] = dict(t = 40, b = 40, r = 40, l = 40)
    
    if stop_legend:
        fig['data'].append(dict(x=df['Date_2'], y=df['ATR_sl_long'], type='scatter', yaxis='y2', 
                                line = dict(width = 1), marker=dict(color='#FF4500'), name='SL_long'))
        fig['data'].append(dict(x=df['Date_2'], y=df['ATR_sl_short'], type='scatter', yaxis='y2', 
                            line = dict(width = 1), marker=dict(color='#FF4500'), name='SL_short'))
        fig['data'].append(dict(x=df['Date_2'], y=df['ATR_sp_long'], type='scatter', yaxis='y2', 
                                line = dict(width = 1), marker=dict(color='#008B8B'), name='SP_long'))
        fig['data'].append(dict(x=df['Date_2'], y=df['ATR_sp_short'], type='scatter', yaxis='y2', 
                            line = dict(width = 1), marker=dict(color='#008B8B'), name='SP_short'))


    plotly.offline.iplot(fig)
    return


In [3]:
path = r'C:\Users\parsh\Desktop\Exchange\data\US_stocks_csv'
file_name = '/NYSE/ABX.csv'
file_ticker = path + file_name

vol_df = pd.read_csv(file_ticker)
vol_df['Date'] = pd.to_datetime(vol_df['Date'], format='%Y%m%d', errors='ignore')

# Turtle simple soup

In [4]:
vol_df['max_20_bars'] = vol_df['High'][vol_df['High'].rolling(20).max() == vol_df['High']]

vol_df['max_20_bars'] = vol_df['max_20_bars'].fillna(-1e9)



vol_df['max_Nmax'] = vol_df['max_20_bars'].shift(4).rolling(17).max()

In [52]:
vol_df['max_Nmax'] = vol_df['max_20_bars'].shift(4).rolling(17).max()

vol_df['max_3_days'] = vol_df['High'].shift(1).rolling(3).max()

vol_df['short_signal'] = ((vol_df['High'] > vol_df['max_Nmax']) & \
                        (vol_df['max_Nmax'] > vol_df['max_3_days']) & \
                        (vol_df['max_Nmax'] > vol_df['Close'])) * (-1)

In [44]:
vol_df['max_20_days'] = vol_df['High'].shift(1).rolling(20).max()

vol_df['false_break_1_condition_1'] = (vol_df['High'] > vol_df['max_20_days']) & (vol_df['max_20_days'] > vol_df['Close'])

vol_df['max_3_days'] = vol_df['High'].shift(1).rolling(3).max()

vol_df['false_break_1_condition_2'] = vol_df['max_20_days'] > vol_df['max_3_days']

vol_df['false_break_1'] = vol_df['false_break_1_condition_1'] & vol_df['false_break_1_condition_2']

In [45]:
vol_df[(vol_df['short_signal'] == -1) & (vol_df['false_break_1'] == False) ]

,Ticker,Date,Open,High,Low,Close,Volume,max_20_bars,max_Nmax,max_3_days,short_signal,Date_2,max_20_days,false_break_1_condition_1,false_break_1_condition_2,false_break_1


In [53]:
vol_df[(vol_df['false_break_1'] == True) & (vol_df['short_signal'] == 0)].head()

,Ticker,Date,Open,High,Low,Close,Volume,max_20_bars,max_Nmax,max_3_days,short_signal,Date_2,max_20_days,false_break_1_condition_1,false_break_1_condition_2,false_break_1
767,ABX,1988-02-17,5.120770,5.151990,5.027090,5.027090,556059,5.151990,-1000000000.000000,5.089540,0,17-02-1988,5.120770,True,True,True
1987,ABX,1992-12-11,14.625000,14.750000,14.562500,14.562500,418200,14.750000,-1000000000.000000,14.625000,0,11-12-1992,14.687500,True,True,True
2310,ABX,1994-03-23,25.000000,25.500000,24.750000,25.250000,1211900,25.500000,-1000000000.000000,25.125000,0,23-03-1994,25.375000,True,True,True
2421,ABX,1994-08-31,22.500000,22.875000,22.375000,22.625000,2437300,22.875000,-1000000000.000000,22.625000,0,31-08-1994,22.750000,True,True,True
3097,ABX,1997-05-05,23.250000,24.250000,23.000000,23.250000,913300,24.250000,-1000000000.000000,22.875000,0,05-05-1997,24.125000,True,True,True


In [54]:
vol_df[vol_df['Date']== '1997-05-05']

,Ticker,Date,Open,High,Low,Close,Volume,max_20_bars,max_Nmax,max_3_days,short_signal,Date_2,max_20_days,false_break_1_condition_1,false_break_1_condition_2,false_break_1
3097,ABX,1997-05-05,23.250000,24.250000,23.000000,23.250000,913300,24.250000,-1000000000.000000,22.875000,0,05-05-1997,24.125000,True,True,True


In [51]:
vol_df.tail(40)

,Ticker,Date,Open,High,Low,Close,Volume,max_20_bars,max_Nmax,max_3_days,short_signal,Date_2,max_20_days,false_break_1_condition_1,false_break_1_condition_2,false_break_1
8503,ABX,2018-10-25,12.900000,13.180000,12.510000,12.570000,31091732,-1000000000.000000,13.410000,14.100000,0,25-10-2018,14.100000,False,False,False
8504,ABX,2018-10-26,12.700000,13.270000,12.670000,12.850000,24605804,-1000000000.000000,13.410000,14.100000,0,26-10-2018,14.100000,False,False,False
8505,ABX,2018-10-29,12.770000,12.970000,12.600000,12.680000,15746723,-1000000000.000000,14.100000,13.440000,0,29-10-2018,14.100000,False,True,False
8506,ABX,2018-10-30,12.600000,12.780000,12.415000,12.710000,13378868,-1000000000.000000,14.100000,13.270000,0,30-10-2018,14.100000,False,True,False
8507,ABX,2018-10-31,12.650000,12.660000,12.320000,12.550000,23462988,-1000000000.000000,14.100000,13.270000,0,31-10-2018,14.100000,False,True,False
8508,ABX,2018-11-01,12.820000,13.450000,12.780000,13.340000,21163172,-1000000000.000000,14.100000,12.970000,0,01-11-2018,14.100000,False,True,False
8509,ABX,2018-11-02,13.220000,13.335000,13.010000,13.170000,12971645,-1000000000.000000,14.100000,13.450000,0,02-11-2018,14.100000,False,True,False
8510,ABX,2018-11-05,13.170000,13.460000,13.070000,13.210000,13967423,-1000000000.000000,14.100000,13.450000,0,05-11-2018,14.100000,False,True,False
8511,ABX,2018-11-06,13.160000,13.220000,12.990000,13.140000,9400718,-1000000000.000000,14.100000,13.460000,0,06-11-2018,14.100000,False,True,False
8512,ABX,2018-11-07,13.240000,13.290000,12.960000,12.980000,10592891,-1000000000.000000,14.100000,13.460000,0,07-11-2018,14.100000,False,True,False


In [46]:
vol_df[(vol_df['false_break_1'] == True)].shape

(94, 16)

In [47]:
vol_df[(vol_df['short_signal'] == -1)].shape

(75, 16)

In [22]:
plot_signal(vol_df, signal_name = 'false_break_1')